# Step 1 - Climate Analysis and Exploration

In [ ]:
# Dependencies and setup
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc

In [ ]:
# Create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Produce an inspection object for the given target
inspector = inspect(engine)

In [ ]:
# Use inspector to print the column names and types from measurement table
m = inspector.get_columns("measurement")
for column in m:
    print(column["name"], column["type"])

In [ ]:
# Use inspector to print the column names and types from stations table
s = inspector.get_columns("station")
for column in s:
    print(column["name"], column["type"])

### Precipitation Analysis

In [ ]:
# Find the most recent date in the dataset
stringRecent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]
stringRecent_date

In [ ]:
# Convert string date format to date timestamp format for datetime module
mostRecent_date = (dt.datetime.strptime(stringRecent_date, "%Y-%m-%d")).date()
mostRecent_date

In [ ]:
# Calculate the date one year prior to the most recent date in dataset
mostFormer_date = mostRecent_date - dt.timedelta(days = 365)
mostFormer_date

In [ ]:
# Perform a query to retrieve the dates and precipitation scores
precipitationData = session.query(measurement.date, measurement.prcp).\
                    filter((measurement.date <= mostRecent_date) & (measurement.date >= mostFormer_date)).all()

In [ ]:
# Save the query results as a pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitationData)
precipitation_df = precipitation_df.set_index("date")

In [ ]:
# Sort the DataFrame by date
precipitation_df = precipitation_df.sort_index()

In [ ]:
# Drop rows containing NaN values
precipitation_df.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = True)

In [ ]:
# Rename column for cleaner DataFrame and for plot legend
precipitation_df = precipitation_df.rename(columns = {"prcp":"Precipitation"})
precipitation_df.head()

In [ ]:
# Define plot size, color and transparency, legend, and x tickmark orientation
pandasPlot = precipitation_df.plot(figsize = (18,10), color = "mediumseagreen", alpha = 1, legend = True, rot = 45)

# Define plot title, x label, and y label (and their font sizes)
pandasPlot.set_title(f"Daily Precipitation Totals ({mostFormer_date} thru {mostRecent_date})", fontsize = 18)
pandasPlot.set_xlabel("Date", fontsize = 14)
pandasPlot.set_ylabel("Precipitation (in inches)", fontsize = 14)

# Print plot to image file
plt.savefig("Images/Daily_Precipitation.jpg")

# Display plot
plt.show()

In [ ]:
# Use pandas to calculate the summary statistics for the precipitation data
precipitation_df.describe()

### Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
totalStations = session.query(station).count()

# Print total number of stations to display
print(f"The total number of stations in the dataset is:  {totalStations}.")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
mostActive_stations = session.query(measurement.station, func.count(measurement.station), station.name).\
                      order_by(func.count(measurement.station).desc()).\
                      group_by(measurement.station).all()

# List the stations and the counts in descending order
mostActive_stations

In [ ]:
# Declare variables to store extracted values from array
mostActive_id = mostActive_stations[0][0]
mostActive_name = mostActive_stations[0][2]

# Print station with highest number of observations to display
print(f"Station ID {mostActive_id} ({mostActive_name}) has the highest number of observations.")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature
lowest_temp = session.query(measurement.tobs).\
              filter(measurement.station == mostActive_id).\
              order_by(measurement.tobs.asc()).first()

highest_temp = session.query(measurement.tobs).\
               filter(measurement.station == mostActive_id).\
               order_by(measurement.tobs.desc()).first()

average_temp = session.query(func.avg(measurement.tobs)).\
               filter(measurement.station == mostActive_id)

# Print lowest, average, and highest temperatures to display
print(f"At station ID {mostActive_id} ({mostActive_name}), the lowest temperature recorded was {lowest_temp[0]}°F.")
print(f"At station ID {mostActive_id} ({mostActive_name}), the average temperature recorded was {average_temp[0]}°F.")
print(f"At station ID {mostActive_id} ({mostActive_name}), the highest temperature recorded was {highest_temp[0]}°F.")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperatureData = session.query(measurement.tobs).\
                  filter((measurement.station == mostActive_id)\
                  & (measurement.date <= mostRecent_date)\
                  & (measurement.date >= mostFormer_date)).all()
temperatureData

In [ ]:
# Save the query results as a pandas DataFrame
temperature_df = pd.DataFrame(temperatureData)
temperature_df

In [ ]:
# Rename column for cleaner DataFrame and for plot legend
temperature_df = temperature_df.rename(columns = {"tobs":"Temperature Observation"})
temperature_df.head()

In [ ]:
# Pass values into histogram plot, and define color, transparency, plot size, and number of bins
temperature_df.plot.hist(color = "darkorange", ec = "k", alpha = 0.5, figsize = (18, 10), bins = 12)

# Define histogram plot title, x label, and y label (and their font sizes)
plt.title(f"Frequency of Recorded Temperatures at Station ID {mostActive_id} ({mostActive_name})", fontsize = 18)
plt.xlabel("Temperature (F°)", fontsize = 14)
plt.ylabel("Number of Observations", fontsize = 14)

# Print histogram plot to image file
plt.savefig("Images/Temperatures_Histogram.jpg")

# Display histogram plot
plt.show()

# Close Session

In [ ]:
# Close session
session.close()